In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP
from dfdb import DFDB

import types
import copy

Using TensorFlow backend.


In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [12]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_colwidth', -1)

In [9]:
def revert_rfe(df_train, param, sorted_columns, df_test, trial, start_columns, limit=None, remark=None):
    
    # init cv_score and try only base feature
    selected_columns = copy.deepcopy(start_columns)
    if type(limit) == type(None):
        limit = len(sorted_columns)
    args = copy.deepcopy(param)
    args['columns'] = selected_columns
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=trial, remark=remark)
    val_mae_mean = np.mean(df_his.valid)
    cv_score = val_mae_mean
    
    # add feature one by one and check cv score change
    for idx,col in enumerate(sorted_columns):
#         if idx in start_column_index:
#             continue
        args = copy.deepcopy(param)
        args['columns'] = list(set(selected_columns + [col]))
        df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=trial, remark=remark)
        val_mae_mean = np.mean(df_his.valid)
        if val_mae_mean < cv_score:
            selected_columns.append(col)
            cv_score = val_mae_mean
        if len(selected_columns) >= limit:
            break
        
    return selected_columns


In [4]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [5]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [6]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [7]:
original_columns = df_train.columns.drop(['index','y','label','group']).tolist()

In [10]:
#check feature_importances
# df_feature_importances = df_trial.loc[294]['df_feature_importances']
# sorted_columns = EP.evaluate(df_feature_importances, key='average_permutation_weight')
# df_feature_importances.sort_values(by=['average_permutation_weight'], ascending=False).head(100)
# len(sorted_columns)

In [8]:
# mytrial = []
db = DFDB('../trial/xgbm.pkl', auto_commit=False)

In [10]:
df_trial = db.select()
df_trial['kfold'] = df_trial['param'].apply(lambda x: x['kfold'])

In [14]:
df_trial[['datetime','nfeatures', 'kfold', 'remark', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']].loc[248:273]

,datetime,nfeatures,kfold,remark,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
248,2019-05-06 04:48:20.973679,1820,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.991750,0.051529,2.054514,0.717275,0.062764,None
249,2019-05-06 05:48:37.256732,22,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,2.009075,0.037867,2.018946,0.661665,0.009871,None
250,2019-05-06 05:48:45.455757,17,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.997562,0.026410,1.971074,0.621503,-0.026489,None
251,2019-05-06 05:52:29.282481,22,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,2.009075,0.037867,2.018946,0.661665,0.009871,None
252,2019-05-06 05:52:40.406186,17,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.997562,0.026410,1.971074,0.621503,-0.026489,None
253,2019-05-06 06:09:20.216642,120,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.995402,0.046402,2.023155,0.673155,0.027753,None
254,2019-05-06 06:09:53.758052,115,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.964263,0.038414,2.019078,0.692620,0.054816,None
255,2019-05-06 06:10:27.488974,110,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.968276,0.037392,2.001287,0.682020,0.033011,None
256,2019-05-06 06:10:58.635970,105,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.987626,0.040535,1.990721,0.654039,0.003095,None
257,2019-05-06 06:11:28.330412,100,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.975875,0.039302,1.984727,0.644857,0.008852,None


In [15]:
param_idx = 273
column_idx = 253
db_ = db
df_trial_ = df_trial
mytrial = []
columns = copy.deepcopy(df_trial_.loc[column_idx]['param']['columns'])
param = copy.deepcopy(df_trial_.loc[param_idx]['param'])
# param['algorithm']['init'] = {'logging_level': 'Silent', **param['algorithm']['init']}
param['scaler']['init'] = {}
param['algorithm']['fit'] = {}

selected_columns = revert_rfe(df_train, param, columns, df_test, mytrial, start_columns=[columns[0]], limit=20, remark='start from top1 column')
print(len(selected_columns))
for trial_i in mytrial:
    db_.insert(trial_i)
df_trial_ = db_.select()
df_trial_['kfold'] = df_trial_['param'].apply(lambda x: x['kfold'])

12


In [18]:
df_trial_[df_trial_['remark']=='start from top1 column'][['datetime','nfeatures', 'kfold', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,kfold,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
1300,2019-05-22 07:45:45.971968,1,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",2.058100,0.000010,2.069261,0.000047,0.011161
1301,2019-05-22 07:45:46.968994,2,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",2.054077,0.000014,2.069573,0.000035,0.015496
1302,2019-05-22 07:45:48.007513,2,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",2.063218,0.000056,2.080734,0.000008,0.017516
1303,2019-05-22 07:45:48.923666,2,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",2.056661,0.000004,2.071492,0.000076,0.014831
1304,2019-05-22 07:45:49.910746,2,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",2.052949,0.000007,2.066940,0.000029,0.013990
1305,2019-05-22 07:45:51.038370,3,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",2.018186,0.000005,2.037143,0.000017,0.018957
1306,2019-05-22 07:45:52.223098,4,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",2.004512,0.000013,2.026882,0.000059,0.022370
1307,2019-05-22 07:45:53.465455,5,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",2.001675,0.000009,2.025475,0.000045,0.023800
1308,2019-05-22 07:45:54.781749,6,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",1.998562,0.000009,2.023556,0.000027,0.024994
1309,2019-05-22 07:45:55.984614,7,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",1.994130,0.000001,2.020952,0.000053,0.026823


In [19]:
db_.commit()

In [20]:
len(selected_columns)

12

In [21]:
selected_columns

['q25_roll_std_100',
 'abs_q25_5',
 'spkt_welch_density__coeff_3',
 'abs_q75_7',
 'spkt_welch_densitycoeff_2',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'agg_autocorrelationf_agg_"mean"__maxlag_40',
 "number_peaks{'n': 1}",
 '5000peak_peak_amp_max_',
 'abs_q95_3',
 'spkt_welch_density__coeff_89',
 'abs_q05_2']

In [27]:
mytrial = []
#  tune hypterparameters
def objective(trial):
        
    max_depth = trial.suggest_int('max_depth', 2, 6)
    max_bin = trial.suggest_int('max_bin', 10, 100)
    eta = trial.suggest_uniform('eta', 0.01, 0.4)
    colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.6, 1)
    min_child_weight = trial.suggest_int('min_child_weight', 200, 600)
    n_estimators = trial.suggest_int('n_estimators', 100, 500)
    subsample = trial.suggest_uniform('subsample', 0.6, 1.0)
    reg_lambda = trial.suggest_loguniform('reg_lambda', 0.000001, 1.0)
    reg_alpha = trial.suggest_loguniform('reg_alpha', 0.000001, 1.0)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':selected_columns,
        'kfold':{
            'n_splits': 8,
            'random_state': 1985,
            'shuffle': True,
            'type': 'group'
        },
        'scaler':{
            'cls':'StandardScaler',
            'init':{}
        },
        'algorithm':{
            'cls':'xgb.XGBRegressor',
            'init':{
                "max_depth":max_depth,
                "max_bin":max_bin,
                "eta":eta,
                "colsample_bytree":colsample_bytree,
                "min_child_weight":min_child_weight,
                "n_estimators":n_estimators,
                "subsample":subsample,
                "reg_lambda":reg_lambda,
                "reg_alpha":reg_alpha,
                'n_jobs':16
            },
            'fit':{
            },
        },
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 1444')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-17 13:54:59,915] Finished trial#0 resulted in value: 0.5232776226557809. Current best value is 0.5232776226557809 with parameters: {'max_depth': 2, 'max_bin': 39, 'eta': 0.21282522960350567, 'colsample_bytree': 0.703711232816592, 'min_child_weight': 534, 'n_estimators': 497, 'subsample': 0.6506789037249469, 'reg_lambda': 0.8090706683463588, 'reg_alpha': 0.8995260266487242, 'random_state': 8464}.
[I 2019-05-17 13:55:03,288] Finished trial#1 resulted in value: 0.6296763956188193. Current best value is 0.5232776226557809 with parameters: {'max_depth': 2, 'max_bin': 39, 'eta': 0.21282522960350567, 'colsample_bytree': 0.703711232816592, 'min_child_weight': 534, 'n_estimators': 497, 'subsample': 0.6506789037249469, 'reg_lambda': 0.8090706683463588, 'reg_alpha': 0.8995260266487242, 'random_state': 8464}.
[I 2019-05-17 13:55:14,962] Finished trial#2 resulted in value: 1.9060395348687709. Current best value is 0.5232776226557809 with parameters: {'max_depth': 2, 'max_bin': 39, 'eta':

[I 2019-05-17 13:57:22,094] Finished trial#38 resulted in value: 0.9083804886898339. Current best value is 0.22783758625832246 with parameters: {'max_depth': 2, 'max_bin': 44, 'eta': 0.16935188599687143, 'colsample_bytree': 0.8433226106360044, 'min_child_weight': 432, 'n_estimators': 101, 'subsample': 0.8740912686634448, 'reg_lambda': 0.4553205099812844, 'reg_alpha': 0.3007988035057162, 'random_state': 4147}.
[I 2019-05-17 13:57:24,844] Finished trial#39 resulted in value: 0.7384698459360912. Current best value is 0.22783758625832246 with parameters: {'max_depth': 2, 'max_bin': 44, 'eta': 0.16935188599687143, 'colsample_bytree': 0.8433226106360044, 'min_child_weight': 432, 'n_estimators': 101, 'subsample': 0.8740912686634448, 'reg_lambda': 0.4553205099812844, 'reg_alpha': 0.3007988035057162, 'random_state': 4147}.
[I 2019-05-17 13:57:32,032] Finished trial#40 resulted in value: 1.06712752606034. Current best value is 0.22783758625832246 with parameters: {'max_depth': 2, 'max_bin': 44, 

[I 2019-05-17 13:59:23,959] Finished trial#76 resulted in value: 0.3745737543477224. Current best value is 0.22783758625832246 with parameters: {'max_depth': 2, 'max_bin': 44, 'eta': 0.16935188599687143, 'colsample_bytree': 0.8433226106360044, 'min_child_weight': 432, 'n_estimators': 101, 'subsample': 0.8740912686634448, 'reg_lambda': 0.4553205099812844, 'reg_alpha': 0.3007988035057162, 'random_state': 4147}.
[I 2019-05-17 13:59:25,630] Finished trial#77 resulted in value: 0.38701482137119947. Current best value is 0.22783758625832246 with parameters: {'max_depth': 2, 'max_bin': 44, 'eta': 0.16935188599687143, 'colsample_bytree': 0.8433226106360044, 'min_child_weight': 432, 'n_estimators': 101, 'subsample': 0.8740912686634448, 'reg_lambda': 0.4553205099812844, 'reg_alpha': 0.3007988035057162, 'random_state': 4147}.
[I 2019-05-17 13:59:27,439] Finished trial#78 resulted in value: 0.30710311650110017. Current best value is 0.22783758625832246 with parameters: {'max_depth': 2, 'max_bin': 

[I 2019-05-17 14:01:20,149] Finished trial#114 resulted in value: 0.8986091806712417. Current best value is 0.22783758625832246 with parameters: {'max_depth': 2, 'max_bin': 44, 'eta': 0.16935188599687143, 'colsample_bytree': 0.8433226106360044, 'min_child_weight': 432, 'n_estimators': 101, 'subsample': 0.8740912686634448, 'reg_lambda': 0.4553205099812844, 'reg_alpha': 0.3007988035057162, 'random_state': 4147}.
[I 2019-05-17 14:01:23,834] Finished trial#115 resulted in value: 0.789268346573917. Current best value is 0.22783758625832246 with parameters: {'max_depth': 2, 'max_bin': 44, 'eta': 0.16935188599687143, 'colsample_bytree': 0.8433226106360044, 'min_child_weight': 432, 'n_estimators': 101, 'subsample': 0.8740912686634448, 'reg_lambda': 0.4553205099812844, 'reg_alpha': 0.3007988035057162, 'random_state': 4147}.
[I 2019-05-17 14:01:25,540] Finished trial#116 resulted in value: 0.29889706599204396. Current best value is 0.22783758625832246 with parameters: {'max_depth': 2, 'max_bin':

[I 2019-05-17 14:02:56,891] Finished trial#152 resulted in value: 0.32495653310728495. Current best value is 0.22121668620719104 with parameters: {'max_depth': 2, 'max_bin': 35, 'eta': 0.0932003472753572, 'colsample_bytree': 0.784639839428384, 'min_child_weight': 365, 'n_estimators': 100, 'subsample': 0.6502017167178653, 'reg_lambda': 0.30072802701797746, 'reg_alpha': 0.4657245495732788, 'random_state': 5895}.
[I 2019-05-17 14:02:58,129] Finished trial#153 resulted in value: 0.23122367743262698. Current best value is 0.22121668620719104 with parameters: {'max_depth': 2, 'max_bin': 35, 'eta': 0.0932003472753572, 'colsample_bytree': 0.784639839428384, 'min_child_weight': 365, 'n_estimators': 100, 'subsample': 0.6502017167178653, 'reg_lambda': 0.30072802701797746, 'reg_alpha': 0.4657245495732788, 'random_state': 5895}.
[I 2019-05-17 14:03:00,007] Finished trial#154 resulted in value: 0.309905258187627. Current best value is 0.22121668620719104 with parameters: {'max_depth': 2, 'max_bin': 

[I 2019-05-17 14:04:27,174] Finished trial#190 resulted in value: 0.47238339800433954. Current best value is 0.22121668620719104 with parameters: {'max_depth': 2, 'max_bin': 35, 'eta': 0.0932003472753572, 'colsample_bytree': 0.784639839428384, 'min_child_weight': 365, 'n_estimators': 100, 'subsample': 0.6502017167178653, 'reg_lambda': 0.30072802701797746, 'reg_alpha': 0.4657245495732788, 'random_state': 5895}.
[I 2019-05-17 14:04:28,952] Finished trial#191 resulted in value: 0.2765353224833394. Current best value is 0.22121668620719104 with parameters: {'max_depth': 2, 'max_bin': 35, 'eta': 0.0932003472753572, 'colsample_bytree': 0.784639839428384, 'min_child_weight': 365, 'n_estimators': 100, 'subsample': 0.6502017167178653, 'reg_lambda': 0.30072802701797746, 'reg_alpha': 0.4657245495732788, 'random_state': 5895}.
[I 2019-05-17 14:04:30,774] Finished trial#192 resulted in value: 0.3553107148493051. Current best value is 0.22121668620719104 with parameters: {'max_depth': 2, 'max_bin': 

In [29]:
for trial_i in mytrial:
    db.insert(trial_i)

In [42]:
df_trial = db.select()
df_trial[(df_trial['remark']=='tune 51')&(df_trial['mae_diff']<.11)][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
183,2019-05-17 14:02:01.242754,tune 51,51,1.949187,0.003206,2.056744,0.01394,0.107557


In [43]:
db.commit()